Hypothetical interaction

In [1]:
import sys
import os

# Add actrme to path
sys.path.append(os.path.abspath(os.path.join(os.path.abspath("."), '..', 'actrme')))
from importlib import reload
import actrme.models.model
import actrme.basic
from actrme.modules.declarative import DeclarativeMemory
import pandas as pd

reload(actrme.basic)
reload(actrme.models.model)


<module 'actrme.models.model' from '/Users/andreastocco/Documents/Research/ACTR-ME/actrme/models/model.py'>

# Examples

Here is a list of examples.

## 1. A Memory Experiment

We will start with a memory experoiment. The experiment is taken from a paper by Xu et al. (2025), and the original data is available on OSF.

The experiment used a retrieval practice procedure: Participants are presented with simple paired associates, e.g., a pair of Swahili-English words, such as "samaki/fish". Each pair is presented during a study trial, and then probed in a series of test trials.

In [2]:
data = pd.read_csv("../data/memory_experiment.csv")
data

,subj,item,resp,RT,type,time,acc
0,1501,3,fish,2.295,study,3.131,1
1,1501,3,fish,0.888,test,5.426,1
2,1501,12,book,3.128,study,10.248,1
3,1501,12,book,0.841,test,12.776,1
4,1501,2,beer,4.203,study,18.824,1
...,...,...,...,...,...,...,...
146,1501,10,cloud,2.137,test,712.918,1
147,1501,5,office,1.025,test,715.292,1
148,1501,12,book,0.975,test,717.400,1
149,1501,16,work,1.042,test,719.503,1


We first need to add two columns where the model will store its predictions

In [3]:
data['predicted_resp'] = ""
data['predicted_rt'] = 0.0

The dataset is not quite ready for analysis yet. To run this analysis, we need to add a column that contains the correct response for each item. To do so, we first need to create a dictoriary of all the responses for each 'study' trial; these are the correct pair-associates.

In [4]:
data['item'] = data['item'].astype(str)
study = data[data['type'] == 'study']
mappings = dict(zip(study.item, study.resp))
mappings

{'3': 'fish',
 '12': 'book',
 '2': 'beer',
 '9': 'police',
 '22': 'fruit',
 '5': 'office',
 '14': 'voice',
 '23': 'pain',
 '8': 'music',
 '10': 'cloud',
 '18': 'friend',
 '11': 'chicken',
 '25': 'language',
 '15': 'thanks',
 '20': 'bike',
 '7': 'eye',
 '16': 'work',
 '19': 'wheel',
 '13': 'song',
 '4': 'pants',
 '21': 'color',
 '6': 'monkey',
 '24': 'shop',
 '1': 'enemy',
 '17': 'holidays'}

Now we can add a new column, 'correct', that contains the correct response for each item presented, whether the participant responded correctly or not.

In [5]:
data['correct'] = data['resp']
for index, row in data.iterrows():
    if row['type'] == "test":
        data.at[index, 'correct'] = mappings[row['item']]
    #elif row['type'] == "study":
    #    data.at[index, 'RT'] = np.nan

data

,subj,item,resp,RT,type,time,acc,predicted_resp,predicted_rt,correct
0,1501,3,fish,2.295,study,3.131,1,,0.0,fish
1,1501,3,fish,0.888,test,5.426,1,,0.0,fish
2,1501,12,book,3.128,study,10.248,1,,0.0,book
3,1501,12,book,0.841,test,12.776,1,,0.0,book
4,1501,2,beer,4.203,study,18.824,1,,0.0,beer
...,...,...,...,...,...,...,...,...,...,...
146,1501,10,cloud,2.137,test,712.918,1,,0.0,cloud
147,1501,5,office,1.025,test,715.292,1,,0.0,office
148,1501,12,book,0.975,test,717.400,1,,0.0,book
149,1501,16,work,1.042,test,719.503,1,,0.0,work


Now, we need to separate test and study trials. Study trials should be encoded, and study trials should be used as cues. To do so, we need to move the 'item' values into a new column, 'cue_item':

In [6]:
data['cue_item'] = data['item']
for index, row in data.iterrows():
    if row['type'] == "test":
        data.at[index, 'item'] = ""
    else:
        data.at[index, 'cue_item'] = ""

data

,subj,item,resp,RT,type,time,acc,predicted_resp,predicted_rt,correct,cue_item
0,1501,3,fish,2.295,study,3.131,1,,0.0,fish,
1,1501,,fish,0.888,test,5.426,1,,0.0,fish,3
2,1501,12,book,3.128,study,10.248,1,,0.0,book,
3,1501,,book,0.841,test,12.776,1,,0.0,book,12
4,1501,2,beer,4.203,study,18.824,1,,0.0,beer,
...,...,...,...,...,...,...,...,...,...,...,...
146,1501,,cloud,2.137,test,712.918,1,,0.0,cloud,10
147,1501,,office,1.025,test,715.292,1,,0.0,office,5
148,1501,,book,0.975,test,717.400,1,,0.0,book,12
149,1501,,work,1.042,test,719.503,1,,0.0,work,16


This is an alteration of "study" and "test" trials. 

### Creating a model

To fit the data, we need to create a simple model. Fitting the data means that the model is run trial by trial to maximize the probability of responding exactly like the participant.

To do so, we need to create a `DataModel`, which is a generic name for a model simulates behavior or performance "tracing".

In [7]:
#reload(actrme.models.model)
model = actrme.models.model.DataModel(data)
model

Now, we have not yet specified how the model works. In this example, we will create a simple model that does only one thing: encodes things on study trials, and attempts to retrieve them on test trials. To do so, we need to add a new cognitive function to the model, Declarative Memory. 

In [8]:
reload(actrme.modules.declarative)
dm = actrme.modules.declarative.DeclarativeMemory()
#print(dm)
model.add_module(dm)
model.add_input(dm.get_input("encode"))
model.add_input(dm.get_input("cue"))
model.add_output(dm.get_output("retrieval"))


Now, let's check the model's outputs

In [9]:
print(model.outputs)

[<(Num..) 'rt'=0>, <(Sym..) 'retrieval'={}>]


... And the model's inputs

In [10]:
print(model.inputs)

[<(..Num) 'time'=0>, <(..Sym) 'encode'={}>, <(..Sym) 'cue'={}>]


## Connecting the model to the data.

The last step in preparing a data model is to connect the model's inputs and outputs to the data. In the data, we have separated study and test information in different columns. The study information ('item' and 'resp') flows into the model's 'encode' input, while the test information ('cue_item') flows into the 'cue' input. The model memorizes information when there is information in the 'encode' input, and retrieves information when there is data in the 'cue' inpit.

In [11]:
model.connect_input("item", model.get_input("encode"))
model.connect_input("resp", model.get_input("encode"))
model.connect_input("cue_item", model.get_input("cue"))
model.connect_input("time", model.get_input("time"))

Now, we want to save the model's response and response times.

In [12]:
# We want to simply maximize the probability of a response

model.connect_output("predicted_resp", model.get_output("retrieval"), "resp")

## Running the model and analyzing the data

In [13]:
# Now, run the model
reload(actrme.basic)
model.run()


input owned by <actrme.models.model.DataModel object at 0x1272ca610>)
input owned by <Declarative Memory [module]>)


AssertionError: Not a valid IO object: <(..Sym) 'retrieval'={}>

In [15]:
isinstance(dm.get_input("encode"), actrme.basic.SymbolicIO)

False

In [76]:
data

,subj,item,resp,RT,type,time,acc,predicted_resp,predicted_rt,correct
0,1501,3,fish,2.295,study,3.131,1,,0.0,fish
1,1501,3,fish,0.888,test,5.426,1,,0.0,fish
2,1501,12,book,3.128,study,10.248,1,,0.0,book
3,1501,12,book,0.841,test,12.776,1,,0.0,book
4,1501,2,beer,4.203,study,18.824,1,,0.0,beer
...,...,...,...,...,...,...,...,...,...,...
146,1501,10,cloud,2.137,test,712.918,1,,0.0,cloud
147,1501,5,office,1.025,test,715.292,1,,0.0,office
148,1501,12,book,0.975,test,717.400,1,,0.0,book
149,1501,16,work,1.042,test,719.503,1,,0.0,work
